In [176]:
import os, time
from influxdb_client import InfluxDBClient, QueryApi
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
import time

In [177]:
token = os.environ.get("INFLUXDB_TOKEN")
org = "th-koeln"
url = "http://localhost:8086"
bucket = "iot_telemetry_data"

# Queries

In [178]:
# Query to get all timestamps from the IoT telemetry data to determine data coverage
times_stamps_query = '''
from(bucket: "iot_telemetry_data")
  |> range(start: 0)
  |> filter(fn: (r) => r._measurement == "iot_telemetry")
  |> keep(columns: ["_time"])
'''

In [179]:
# Calculate average humidity per device across all time periods
average_humidity_per_device_query = """
from(bucket: "iot_telemetry_data")
  |> range(start: 0)
  |> filter(fn: (r) =>
      r._measurement == "iot_telemetry" and
      r._field == "humidity"
  )
  |> pivot(
      rowKey:   ["device"],
      columnKey: ["_field"],
      valueColumn: "_value"
  )
  |> mean(column: "humidity")
  |> sort(columns: ["device"])
"""

In [180]:
# Find the highest temperature recorded across all devices and time periods
highest_temp_query = """
from(bucket: "iot_telemetry_data")
  |> range(start: 0)
  |> filter(fn: (r) =>
      r._measurement == "iot_telemetry" and
      r._field == "temp"
  )
  |> group()
  |> max(column: "_value")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
"""

In [181]:
# Calculate total air quality score (CO + LPG + Smoke) per device and rank by worst air quality
worst_air_quality_per_device_query = """
from(bucket: "iot_telemetry_data")
  |> range(start: 0)
  |> filter(fn: (r) =>
      r._measurement == "iot_telemetry" and
      (r._field == "co" or r._field == "lpg" or r._field == "smoke")
  )
  |> pivot(
      rowKey:   ["device", "_time"],
      columnKey:["_field"],
      valueColumn: "_value"
  )
  |> map(fn: (r) => ({
      device: r.device,
      air_quality_score: r.co + r.lpg + r.smoke
  }))
  |> group(columns:["device"])
  |> sum(column: "air_quality_score")
  |> sort(columns: ["air_quality_score"], desc: true)
"""

In [182]:
# Calculate average temperature to humidity ratio across all devices within a specific time window
avg_temp_humidity_ratio_query = """
from(bucket: "iot_telemetry_data")
    |> range(start: 2020-07-14T00:01:00Z, stop: 2020-07-19T03:01:00Z)
    |> filter(fn: (r) => r._measurement == "iot_telemetry" and (r._field == "temp" or r._field == "humidity"))
    |> pivot(rowKey:["_time", "device"], columnKey: ["_field"], valueColumn: "_value")
    |> filter(fn: (r) => exists r.temp and exists r.humidity and r.humidity != 0.0)
    |> map(fn: (r) => ({ r with temp_humidity_ratio: r.temp / r.humidity }))
    |> mean(column: "temp_humidity_ratio")
    |> yield(name: "avg_temp_humidity_ratio")
"""

In [183]:
# Calculate minute-by-minute temperature to humidity ratio per device for time-series analysis
avg_temp_humidity_ratio_per_min_influx_query = """from(bucket: "iot_telemetry_data")
    |> range(start: 2020-07-14T00:01:00Z, stop: 2020-07-19T03:01:00Z)
    |> filter(fn: (r) => r._measurement == "iot_telemetry" and (r._field == "temp" or r._field == "humidity"))
    |> pivot(rowKey:["_time", "device"], columnKey: ["_field"], valueColumn: "_value")
    |> filter(fn: (r) => exists r.temp and exists r.humidity and r.humidity != 0.0)
    |> map(fn: (r) => ({ _time: r._time, device: r.device, _value: r.temp / r.humidity }))
    |> group(columns:["device"])
    |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
    |> yield(name: "avg_temp_humidity_ratio_per_min")
"""


## Helper

# Performance Analysis

In [184]:
query_times = {}

In [185]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(average_humidity_per_device_query)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Average Humidity'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 0.2718780040740967 seconds.


In [186]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(highest_temp_query)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Highest Temp'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 0.028052330017089844 seconds.


In [187]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(worst_air_quality_per_device_query)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Worst Air'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 1.6559348106384277 seconds.


In [188]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(avg_temp_humidity_ratio_query)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Avg Temp Humidity Ratio'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 2.4287164211273193 seconds.


In [189]:
try:
    client = InfluxDBClient(url=url, token=token, org=org)
    query_api = QueryApi(influxdb_client=client)
    start_time = time.time()
    result = query_api.query_data_frame(avg_temp_humidity_ratio_per_min_influx_query)
    end_time = time.time()
    print(f"Query took {end_time - start_time} seconds.")
    query_times['Avg Temp Humidity Ratio per Minite'] = (end_time - start_time)
except InfluxDBError as e:
    print('Error: ', e)

Query took 3.142120599746704 seconds.


# Summary

In [190]:
print(query_times)

{'Average Humidity': 0.2718780040740967, 'Highest Temp': 0.028052330017089844, 'Worst Air': 1.6559348106384277, 'Avg Temp Humidity Ratio': 2.4287164211273193, 'Avg Temp Humidity Ratio per Minite': 3.142120599746704}


In [191]:
performance = "InfluxDB query times: " + str(query_times)

In [192]:
filename = "performance.txt"


# 'a' öffnet die Datei im Append-Modus (erstellt sie, falls sie nicht existiert)
with open(filename, "a", encoding="utf-8") as f:
    f.write(performance + "\n")

print(performance)


InfluxDB query times: {'Average Humidity': 0.2718780040740967, 'Highest Temp': 0.028052330017089844, 'Worst Air': 1.6559348106384277, 'Avg Temp Humidity Ratio': 2.4287164211273193, 'Avg Temp Humidity Ratio per Minite': 3.142120599746704}
